### (0) IMPORT

In [213]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#BORROWING KAISHENGS CLIENT SECRET AND ID THANKS KS BB PLS DONT DISABLE YA
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

### (1) Playlist extractor - Make sure you do not already have a file called "User_Playlist.csv". if you do, please delete it if not this code will encounter an error

In [200]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import os.path as os
import time

# def extract_playlistID(playlists,find_playlist):
#     while True:
#         for playlist in playlists['items']:
#         ##Loop and return playlistID if found
#             if playlist["name"] == find_playlist:
#                 return playlist['id']
#         ##Show next 50 songs
#         if playlists['next']:
#             playlists = sp.next(playlists)
#         else:
#             return None
        
def extract_trackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_nexttrackID(playlist_tracks):
    trackID_list = []
    for tracks in playlist_tracks["items"]:
        track_ID = tracks["track"]["id"]
        trackID_list.append(track_ID)
    return trackID_list

def extract_songInfo(song):
    info = sp.track(song)
    ##Define features u wanna extract here
    id = song
    songName = info["name"]
    albumName = info["album"]["name"]
    albumID = info["album"]["id"]
    artists = info["artists"][0]["name"]
    for artist in info["artists"]:
        artist_genre = sp.artist(artist["id"])
        genre = artist_genre["genres"]
    explicit = info["explicit"]
    releaseDate = info["album"]["release_date"]
    return [id,songName,albumName,albumID,artists,genre,explicit,releaseDate]

def extract_features(song):
    info = sp.audio_features(song)[0]
    ##Define features u wanna extract here
    danceability = info["danceability"]
    energy = info["energy"]
    key = info["key"]
    loudness = info["loudness"]
    mode = info["mode"]
    speechiness = info["speechiness"]
    acousticness = info["acousticness"]
    instrumental = info["instrumentalness"]
    liveness = info["liveness"]
    valence = info["valence"]
    tempo = info["tempo"]
    duration_ms = info["duration_ms"]
    time_signature = info["time_signature"]
    return [danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,
            valence,tempo,duration_ms,time_signature]

#Auth Stuff
auth_manager = SpotifyClientCredentials(client_id="21ff73a9b5a94ea8b3a969b906baead1", client_secret="3761e7947ef542149467196a07cf2563")
sp = spotipy.Spotify(auth_manager=auth_manager)

##INDICATE THE PLAYLIST U WANNA ADD FROM THE USER (IN THIS CASE SPOTIFY IS THE USER AND TOP 50 IS THE PLAYLIST)
# user = "spotify"
# playlists = sp.user_playlists(user, limit=50)

## Hide this if you are inserting your own playlist ID
# for find_playlist in playlists_name:
#     found_ID = extract_playlistID(playlists, find_playlist)
#     if found_ID != None:
#         playlists_ID.append(found_ID)
    
playlists_name = []
playlists_ID = []

## Ask to insert playlist ID
while True:
    url = input("Insert Spotify Playlist ID (Else type 'done'): ")
    if url == "done":
        break
    else:
        playlists_ID.append(url)
    
# Get Playlist Name
print("Obtaining the Following Playlists: ")
for i in playlists_ID:
    playlist_info = sp.playlist(i)
    name = playlist_info["name"]
    print(name)
    playlists_name.append(name)

##Get Info in Playlist with PlaylistID 
tracksID = {}
for i in range(len(playlists_ID)):
    track_list = []
    playlist_tracks = sp.playlist(playlists_ID[i])
    playlist_tracks = playlist_tracks["tracks"]
    track_list = extract_trackID(playlist_tracks)
    for page in range(0,2):
        if playlist_tracks['next']:
            print("More than 100 Songs Detected. WAITT")
            time.sleep(30)
            print("Looking at next 100 songs")
            playlist_tracks = sp.next(playlist_tracks)
            track_list += extract_nexttrackID(playlist_tracks)
    tracksID[playlists_name[i]] = track_list


##Get Other Track Details
results_tocsv = []
results_tocsv.append(["id","songName","albumName","albumID","artist","genre","explicit","releaseDate",
                     "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                    "valence","tempo","duration_ms","time_signature"])
for i in playlists_name:  
    count = 0
    for song in tracksID[i]:
        time.sleep(1)
        song_info = extract_songInfo(song)
        ##Get Audio Features
        song_features = extract_features(song)
        song_info.extend(song_features)
        results_tocsv.append(song_info)
        count += 1
        
        if count == 100:
            print("Trying not to let spotify kill me. 30s wait :D")
            time.sleep(30)
            count = 0

##Use pandas to convert to dataframe
df = pd.DataFrame(results_tocsv[1:], columns = results_tocsv[0])

##Export to CSV
file_name = "User_Playlist.csv"
if os.exists(file_name):
    df1 = pd.read_csv(file_name, encoding="utf_8_sig")
    df2 = df
    df1 = df1.drop(columns=['Unnamed: 0'])
    merged = pd.concat([df1,df2])
    merged = merged.drop_duplicates(ignore_index=True)
    merged.to_csv(file_name, encoding='utf_8_sig')
else:
    df.to_csv(file_name, encoding='utf_8_sig')
print("20s Cooldown...")
time.sleep(20)

Obtaining the Following Playlists: 
Alt test playlist
20s Cooldown...


### (2) FUNCTIONSSS

In [211]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)  
    return tf_df

def get_features_database(databaseDF):
    #Select Features
    databaseDF = databaseDF[["id","songName",
                "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                "valence","tempo","type"]]

    #OHE Features
    key_ohe = ohe_prep(databaseDF, 'key','key') * 0.5
    mode_ohe = ohe_prep(databaseDF, 'mode','mode') * 0.5

    ##Normalise/Scale Audio Columns
    float_cols = databaseDF.dtypes[databaseDF.dtypes == 'float64'].index.values
    floats = databaseDF[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    ##Combine all Features
    final = pd.concat([floats_scaled, key_ohe, mode_ohe, databaseDF["type"]], axis = 1)
    return final

def generate_rec(databaseDF, database_vector, user_vector, genre_top3):
    #Cosine Similarity
    databaseDF["sim"] = cosine_similarity(database_vector,user_vector)
    
    final_reco = []
    # # Drop rows with different genre from top 3 genres
    # rec_top5 = databaseDF[(databaseDF["genre"] == genre_top3[0]) | (databaseDF["genre"] == genre_top3[1]) | (databaseDF["genre"] == genre_top3[2])]

    #Sort and recommend top 5 with same genres
    pre_rec_ordered = databaseDF.sort_values('sim',ascending = False).head(20)

    #Find genre for each db song in order, and append first 5 songs once genre match is found
    for i in range(20):

        track_data = sp.track(pre_rec_ordered["id"].iloc[i])

        artist_ids = []
        for artist in track_data["artists"]:
            artist_ids.append(artist["id"])

            artists_data = sp.artists(artist_ids)

        genres = []
        for artist in artists_data["artists"]:
            ls_art_genre = artist["genres"]

            for j in ls_art_genre: #['funk']
                genres.append(j)

        genres = set(genres) # removes duplicates

        #UNCOMMENT LINE BELOW to show top songs even if none of them share same genre (WHEN THERE ARE NO SONGS RECOMMENDED)
        # print([pre_rec_ordered['artist'].iloc[i], pre_rec_ordered['songName'].iloc[i], pre_rec_ordered['sim'].iloc[i], genres]) 

        for g in genres:
            if g in genre_top3:
                final_reco.append([pre_rec_ordered['artist'].iloc[i], pre_rec_ordered['songName'].iloc[i], pre_rec_ordered['sim'].iloc[i], genres])
                
    return final_reco



### (3) Converting User Playlist Genre info (from spotify api) from str to ls for subsequent explode func to get top 3 playlist genres 
ignore the warning below :)

In [202]:
# (1) Converts all genre ls for each song in df to actual ls so that it can be exploded later -IGNORE THE WARNING
import ast
userDF = pd.read_csv("User_Playlist.csv")
userDF = userDF.drop(columns=["Unnamed: 0"])

# for i in range()
for i in range(len(userDF["genre"])):
    genre = str((userDF['genre'].iloc[i]))
    genre_str = genre.strip("[]")

    last = False

    ls = []
    word = ''

    for ch in genre_str:
        if ch == "'" and last == False: #first '
            last = True

        if last == True and ch not in ["'", ","]:
            word += ch

        if ch == "'" and last == True:
            last == False
            ls.append(word)
            word = ""
    if ' ' in ls:
        ls.remove(' ')
    if '' in ls:
        ls.remove('')
    userDF['genre'][i] = ls

/var/folders/jg/_cjry7m52mq14l8qcqb7mq8r0000gn/T/ipykernel_43387/4196698234.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  userDF['genre'][i] = ls


### (4) Find Top 3 Genres in User Playlist By Countign Frequency of Genres

In [203]:
# (2) Find top 3 genres in playlist
exploded_df = userDF.explode('genre')

freq = exploded_df["genre"].value_counts(sort=True)

genres_count = dict(freq.head(3))
genre_top3 = list(genres_count.keys())
print(genre_top3)
exploded_df

['pop']


,id,songName,albumName,albumID,artist,genre,explicit,releaseDate,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,duration_ms,time_signature
0,7gaA3wERFkFkgivjwbSvkG,"yes, and?","yes, and?",2B9amdrHDIKOoEiPUfZGtb,Ariana Grande,pop,True,2024-01-12,0.788,0.758,...,-6.513,1,0.0505,0.198,0.000085,0.104,0.823,119.008,214994,4


### (5) Extract Both User and Dataset Csv

In [204]:
#Extract Database CSV
databaseDF = pd.read_csv("spotify_data_cleaned.csv", encoding="utf_8_sig")
databaseDF = databaseDF.drop(columns=['Unnamed: 0'])
#Selects columns that we want
databaseDF = databaseDF[["id","artist","songName", "genre","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]]

#Create new feature/column of artist_songName
databaseDF["artist_songName"] = databaseDF["artist"] + "_" + databaseDF["songName"]

#Check for duplicates in database
databaseDF = databaseDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)

In [205]:
#Extract User Playlist CSV
playlistDF = pd.read_csv("User_Playlist.csv", encoding="utf_8_sig")

playlistDF = playlistDF.drop(columns=['Unnamed: 0'])
#Reset Index
playlistDF = playlistDF.reset_index(drop = True)  
#Selects columns that we want
playlistDF = playlistDF[["id","artist","songName","danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]] 

#Create new feature/column of artist_songName
playlistDF["artist_songName"] = playlistDF["artist"] + "_" + playlistDF["songName"]

#Check for duplicates in user Playlist
playlistDF = playlistDF.drop_duplicates(subset=["artist_songName"],ignore_index= True)

### (6) Normalise Features

In [206]:
#Merge user + dataset dataframe to normalise 
#Normalise takes min and max in dataframe as reference and change it to 0 and 1 respectively

#Group the dataframe as we gonna split it again alter
databaseDF["type"] = "Dataset"
playlistDF["type"] = "User"


#Merge the 2 datasets together
combinedDF = pd.concat([databaseDF,playlistDF], ignore_index=True)

#Check for duplicates between user and Database
combinedDF = combinedDF.drop_duplicates(subset=["artist_songName"], keep="last",ignore_index= True)

##Update databaseDF with removed songs from user Playlist
databaseDF = combinedDF[(combinedDF["type"]== "Dataset")]

##Normalise and get Vectors for Dataset + User
normalised_vector = get_features_database(combinedDF)

##Seperate User from databaseDF 
database_vector = normalised_vector[normalised_vector["type"] == "Dataset"]
user_vector = normalised_vector[normalised_vector["type"] == "User"]

#Drop "type" column
database_vector = database_vector.drop(columns="type")
user_vector = user_vector.drop(columns="type")
databaseDF = databaseDF.drop(columns=["type"])


### (7) Merge all User Vector to form single vector (this is where we gotta do sth w clustering model to not squeeze every song vector into one ball of mess bc the recos are so shit)

In [207]:
##Single Vector Creation
final_user_vector_list = []
for i in user_vector.columns:
    final_user_vector_list.append(user_vector[i].sum()/len(user_vector[i]))

#Putting into a vector dataframe
final_user_vector = pd.DataFrame(columns=user_vector.columns,)
final_user_vector.loc[0] = final_user_vector_list

final_user_vector

,danceability,energy,loudness,speechiness,acousticness,instrumental,liveness,valence,tempo,key|0,...,key|4,key|5,key|6,key|7,key|8,key|9,key|10,key|11,mode|0,mode|1
0,0.158871,0.1516,0.159258,0.010402,0.039759,0.000017,0.0208,0.1646,0.095245,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5


### (LAST) RECOMMEND TIME!
if the database songs that have high cosine similarity as input song DO NOT SHARE SAME GENRE (ALL FROM SPOTIFY BTW), THEN WE WILL GET NO RECOS REGARDLESS

SO if recommendation comes back as empty list, go to function and uncomment the print func to see what the best songs our database has (ARTIST, SONG, GENRES, COSINE SIMI SCORE)


In [212]:
##Generate Recc Songs
result = generate_rec(databaseDF,database_vector,final_user_vector, genre_top3)
result

[['TWICE',
  'Missing U',
  0.999784142466076,
  {'k-pop', 'k-pop girl group', 'pop'}],
 ['Rihanna',
  'Dancing In The Dark - From The "Home" Soundtrack',
  0.9997721713896485,
  {'barbadian pop', 'pop', 'urban contemporary'}],
 ['Jess Glynne',
  'No Rights No Wrongs',
  0.9997493413135223,
  {'dance pop', 'pop', 'uk pop'}]]